This notebook shows poor performance of tabular model for time-series.

In [ ]:
!pip install -q fastai2

In [ ]:
from fastai2.tabular.all import *

In [ ]:
input_dir = Path("/kaggle/input/liverpool-ion-switching")

In [ ]:
train_df = pd.read_csv(input_dir/"train.csv")
test_df  = pd.read_csv(input_dir/"test.csv")

In [ ]:
train_df.head()

In [ ]:
test_df.tail()

# Prepare data

In [ ]:
size = 500_000

def prepare(df):
    start = 0
    for b in range(size, len(df)+1, size):
        df.loc[start:b-1,"time"] = list(range(size))
        start += size

In [ ]:
prepare(train_df)

In [ ]:
prepare(test_df)

In [ ]:
train_df.tail()

# Train

In [ ]:
BS=50_000

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(42)

In [ ]:
# cont_names = ['time', 'signal']
# procs = [Categorify, FillMissing, Normalize]
# splits = RandomSplitter(seed=43)(range_of(train_df))
# to = TabularPandas(train_df, procs, cont_names, y_names="open_channels", splits=splits, y_block=CategoryBlock)

In [ ]:
# dls = TabDataLoader(to, bs=BS, shuffle=False, drop_last=False)

In [ ]:
# dls = to.dataloaders(bs=BS, shuffle_train=False)

In [ ]:
dls.show_batch()

In [ ]:
dls = TabularDataLoaders.from_df(train_df, path=".", y_names="open_channels",
                                 cont_names = ['time', 'signal'], y_block=CategoryBlock,
                                 procs = [Categorify, FillMissing, Normalize], bs=BS,
                                 shuffle_train=False)

In [ ]:
# dls.show_batch()

In [ ]:
learn = tabular_learner(dls, metrics=F1Score(average="macro"))

In [ ]:
# learn.lr_find()

In [ ]:
lr = 3e-3

In [ ]:
learn.fit_one_cycle(10, lr_max=lr)

In [ ]:
learn.fit_one_cycle(10, lr_max=lr/10)

# Submission

In [ ]:
dl = learn.dls.test_dl(test_df)

In [ ]:
preds, _ = learn.get_preds(dl=dl)

In [ ]:
subm = pd.read_csv(input_dir/"sample_submission.csv")

In [ ]:
subm["open_channels"] = preds.argmax(1)

In [ ]:
subm.to_csv("submission.csv", float_format='%0.4f', index=False)

In [ ]:
pd.read_csv("submission.csv")